# How to Generate L1 and L2 WFI Files with Roman I-Sim

***

## Kernel Information

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

## Imports
 Libraries used
- *argparse* for formatting input options in romanisim
- *astroquery.gaia* for querying the Gaia catalog
- *astropy.coordinates* for storing celestial coordinates as Python objects
- *astropy.time* for storing time information as Python objects
- *astropy.table* for working with Astropy Table objects
- *galsim* for image simulations
- *numpy* for array operations
- *romanisim* for image simulations
- *s3fs* for accessing files in the RSP S3 bucket

Additionally, we include an optional import of the `concurrent` module to use in the advanced, parallelized example at the end of the tutorial.

In [ ]:
import argparse
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.table import vstack
import galsim
import numpy as np
from romanisim import gaia, bandpass, catalog, log, wcs, persistence, parameters, ris_make_utils as ris
import s3fs

## Introduction
The purpose of this notebook is to show how to generate simulated Level 1 (L1) and Level 2 (L2) Roman WFI ASDF files data with `romanisim`. Details about the Roman data levels can be found [here](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products). A L1 file contains a single uncalibrated ramp exposure in units of Data Numbers (DN).  L1 files are three-dimensional data cubes, one dimension for time and two dimensions for image coordinates, that are shaped as  arrays with (N resultants, 4096 image rows, 4096 image columns). The L2 WFI data are calibrated images in instrumental units of DN / second.  They are two-dimensional arrays shaped as (4088 image rows, 4088 image columns).

**Note:** Roman I-Sim provides several shell scripts that are designed to be convenient for users. However, these shell scripts are not programmatically accessible in Jupyter or with the Python interpreter. We present them below, but will also demonstrate how to use the `romanisim` package in Python.

***

## Tutorial Data

In this tutorial, we will create necessary data in memory or retrieve it from a catalog service. Catalog files are also available in the RSP S3 bucket, and can be streamed into memory using `astropy.table.Table` and the `s3fs` package instructions in the Data Discovery and Access tutorial. Also see the RSP documentation for more information on the catalogs available in the S3 bucket.

## Source Catalog Generation

The `romanisim` package offers two options for generating source catalogs:
1. Retrieve the source catalog from Gaia; or
2. Parametrically generate a catalog of stars and/or galaxies.

First, let's explore how to create a `romanisim`-compatible source catalog using Gaia. We will use a combination of `astroquery` and `romanisim` to query the Gaia catalog and then write the file in a format compatible with `romanisim`.

In our example below, we will query the Gaia DR3 catalog for sources centered at (RA, Dec) = (80.0, 30.0) degrees and within a radius of 1 degree.

**Note:** The Gaia query may take several minutes to complete.

In [ ]:
ra = 80.0  # Right ascension in degrees
dec = 30.0  # Declination in degrees
radius = 1.0  # Search radius in degrees

query = f'SELECT * FROM gaiadr3.gaia_source WHERE distance({ra}, {dec}, ra, dec) < {radius}'
job = Gaia.launch_job_async(query)
# print(job)
result = job.get_results()

Once we have the result from the Gaia query, we can transform it into a format compatible with Roman I-Sim. We can also optionally write it to an Enhanced Character-Separated Value (ECSV) file compatible with Roman I-Sim:

In [ ]:
obs_time = '2026-10-31T00:00:00'
gaia_catalog = gaia.gaia2romanisimcat(result, Time(obs_time), fluxfields=set(bandpass.galsim2roman_bandpass.values()))

# gaia_catalog.write('gaia_catalog.ecsv')

Uncommenting the line in the cell above will save the catalog to disk, but we don't need to do it if we are going to use it in memory.

Now that we have a catalog, let's take a look at it. The catalog in memory is an astropy.table.Table object with over 1e5 rows:

In [ ]:
gaia_catalog

Alternatively, we can generate a completely synthetic catalog of stars and galaxies using tools in Roman I-Sim (see parameters in the cell below):

In [ ]:
# Galaxy catalog parameters

ra = 0.0  # Right ascension of the catalog center in degrees
dec = 0.0  # Declination of the catalog center in degrees
radius = 0.4  # Radius of the catalog in degrees
n_gal = 10_000  # Number of galaxies
faint_mag = 22  # Faint magnitude limit of simulated sources
hlight_radius = 0.3  # Half-light radius at the faint magnitude limit in units of arcseconds
optical_element = ['F106']  # List of optical elements to simulate
seed = 5346  # Random number seed for reproducibility

# Additional star catalog parameters

n_star = 30_000  # Number of stars

galaxy_cat = catalog.make_galaxies(SkyCoord(ra, dec, unit='deg'), n_gal, radius=radius, index=0.4, faintmag=faint_mag, 
                                   hlr_at_faintmag=hlight_radius, bandpasses=optical_element, rng=None, seed=seed)

star_cat = catalog.make_stars(SkyCoord(ra, dec, unit='deg'), n_star, radius=radius, index=5/3., faintmag=faint_mag, 
                              truncation_radius=None, bandpasses=optical_element, rng=None, seed=seed)

full_catalog = vstack([galaxy_cat, star_cat])

# full_catalog.write('parametric_catalog.ecsv', format='ascii.ecsv', overwrite=True)

As before, we have commented out the line that will write this to disk, and instead have kept it in memory. Below, let's print out the synthetic catalog and take a look:

In [ ]:
full_catalog

We can see galaxies at the top of the stacked catalog (notice type == "SER" for Sersic and values of n (the Sersic index) are not -1, while stars have type == PSF).

## Image Simulation

Here we show how to run the actual simulation using Roman I-Sim. The method for running the simulation for both L1 and L2 data is the same, so we will show an example for L2, and give instructions of how to modify this for L1.

In our example, we are simulating only a single image, so we have set the persistance to the default. Future updates may include how to simulate persistance from multiple exposures.

**Notes:** 

- Roman I-Sim allows the user to either use reference files from CRDS or to use no reference files. This latter mode is not recommended.
- Each detector is simulated separately. We include instructions below for how to parallelize the simulations using the Python `concurrent` package.
- Currently, the simulator does not include the effect of 1/f noise.
- In operations, multi-accumulation (MA) tables may be truncated at specific resultants, but here we show how to simulate the full MA table. MA table 109 contains 10 resultants made up of 44 reads (a total exposure time of approximately 134 seconds) with resultants made up of 1, 2, 3, 4, and 6 resultants with the final resultant containing a single read.

First, if we use the Gaia catalog or another catalog that was not created using Roman I-Sim's parametric catalog generation tools, we need to clean out bad sources. This step can be skipped if we use the parametrically-generated catalog above.

In [ ]:
# bandpass = [f for f in gaia_catalog.dtype.names if f[0] == 'F']
# bad = np.zeros(len(gaia_catalog), dtype='bool')
# for b in bandpass:
#      bad = ~np.isfinite(gaia_catalog[b])
#      if hasattr(gaia_catalog[b], 'mask'):
#           bad |= gaia_catalog[b].mask
#      gaia_catalog = gaia_catalog[~bad]

Next, we run the simulation:

In [ ]:
obs_date = '2026-10-31T00:00:00'  # Datetime of the simulated exposure
sca = 1  # Change this number to simulate different WFI detectors 1 - 18
optical_element = 'F106'  # Optical element to simulate
ma_table_number = 109  # Multi-accumulation (MA) table number...do not recommend to change this as it must match files in CRDS
seed = 7  # Galsim random number generator seed for reproducibility
level = 2  # WFI data level to simulate...1 or 2
cal_level = 'cal' if level == 2 else 'uncal'  # File name extension for data calibration level
filename = f'r0003201001001001004_01101_0001_WFI{sca:02d}_{cal_level}.asdf'  # Output file name on disk. Only change the first part up to _WFI to change the rootname of the file.

# Set other arguments for use in Roman I-Sim. The code expects a specific format for these, so this is a little complicated looking.
parser = argparse.ArgumentParser()
parser.set_defaults(usecrds=True, webbpsf=True, level=level, filename=filename, drop_extra_dq=True)
args = parser.parse_args([])

# Set reference files to None for CRDS
for k in parameters.reference_data:
    parameters.reference_data[k] = None

# Set Galsim RNG object
rng = galsim.UniformDeviate(seed)

# Set default persistance information
persist = persistence.Persistence()

# Set metadata
metadata = ris.set_metadata(date=obs_date, bandpass=optical_element, sca=sca, ma_table_number=ma_table_number)

# Update the WCS info
wcs.fill_in_parameters(metadata, SkyCoord(ra, dec, unit='deg', frame='icrs'), boresight=False, pa_aper=0.0)

# Run the simulation
ris.simulate_image_file(args, metadata, full_catalog, rng, persist)

If we want to simulate an L1 ramp cube, then we can change the level variable above to 1, which will also change the output file name to `*_uncal.asdf`. The rest of the information stays the same.

## Running Roman I-Sim with Shell Scripts

You can also run Roman I-Sim tasks from the command line with executables and shell scripts. These are not easily accessible from a Jupyter notebook, but command line executables romanisim-make-image and romanisim-make-catalog (for making a catalog file from Gaia) exist. The scripts in the S3 bucket provide examples for how to use these. Let's look at one now:

In [ ]:
s3_uri = 's3://roman-sci-test-data-prod-summer-beta-test/ROMANISIM/CATALOGS_SCRIPTS/make_ra0.47_dec0.51_pa0_l1_f129.sh'
fs = s3fs.S3FileSystem()

with fs.open(s3_uri, 'rb') as f:
    for line in f.readlines():
        print(line.decode())

For additional help with the executables, the -h option can be used to get the help text for the input arguments:

In [ ]:
!romanisim-make-catalog -h

In [ ]:
!romanisim-make-image -h

### Advanced Use Case: Parallelized Simulations

Often, we will want to run a simulation using multiple detectors rather than just one at a time. Looping over the above in a serial fashion can take quite a long time, so we want to parallelize the work. In the example below, we will show how to parallelize the procedure with the Python standard library `concurrent` module. Note that there are other, more sophisticated tools such as Dask that can achieve scalable results, and we may cover that in future tutorials.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import wait

At this point, it's helpful to redefine our simulation call above as a single function.

In [ ]:
def run_romanisim(catalog, obs_date = '2026-10-31T00:00:00', sca=1, optical_element='F106', ma_table_number=109, level=2, 
                  filename=f'r0003201001001001004_01101_0001'):

    cal_level = 'cal' if level == 2 else 'uncal'
    filename = f'{filename}_WFI{sca:02d}_{cal_level}.asdf'

    # Set other arguments for use in Roman I-Sim. The code expects a specific format for these, so this is a little complicated looking.
    parser = argparse.ArgumentParser()
    parser.set_defaults(usecrds=True, webbpsf=True, level=level, filename=filename, drop_extra_dq=True)
    args = parser.parse_args([])

    # Set reference files to None for CRDS
    for k in parameters.reference_data:
        parameters.reference_data[k] = None

    # Set Galsim RNG object
    rng = galsim.UniformDeviate(seed)

    # Set default persistance information
    persist = persistence.Persistence()

    # Set metadata
    metadata = ris.set_metadata(date=obs_date, bandpass=optical_element, sca=sca, ma_table_number=ma_table_number)

    # Update the WCS info
    wcs.fill_in_parameters(metadata, SkyCoord(ra, dec, unit='deg', frame='icrs'), boresight=False, pa_aper=0.0)

    # Run the simulation
    ris.simulate_image_file(args, metadata, catalog, rng, persist)

We'll also need to allocate a number of threads to the pool. For this example, let's simulate two detectors at the same time, in this case WFI02 and WFI03 since we already simulated WFI01. This should be scalable based on the computational resources you have available on the RSP. 

**WARNING:** Please be cautious when parallelizing tasks such as Roman I-Sim as it can easily consume all of your RSP resources if handled incorrectly!

We have commented out the lines below. If you want to run the parallelized simulation, uncomment all of the lines in the following code cell. **Note: this code cell will take a long time to run.**

In [ ]:
#with ThreadPoolExecutor(2) as pool:
#    futures = [pool.submit(run_romanisim, full_catalog, **{'sca': x}) for x in [2, 3]]
#    wait(futures)

After running this cell, you may notice that some of the log messages are a bit jumbled. That's because the two simulations are running asynchronously on different threads, so messages from both arrive back to the notebook at slightly different but overlapping times.

## Aditional Resources
- [romanisim](https://romanisim.readthedocs.io/en/latest/index.html)
- [romancal](https://roman-pipeline.readthedocs.io/en/latest/index.html)
- [Roman Documentation](https://roman-docs.stsci.edu)

## About this notebook
**Author:** Sanjib Sharma, Tyler Desjardins  
**Updated On:** 2024-09-27

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 